# Preprocessing Different Modalities with Hugging Face
### Text • Image • Audio

## 1️⃣ Load Tokenizer
هنحمل DistilBERT tokenizer عشان نحول النص لـ Token IDs
## 2️⃣ Text Normalization
بنشوف التوكنایزر بيعمل lowercase وإزالة الرموز إزاي
## 3️⃣ Tokenization + Encoding
تحويل النص لـ input_ids و attention_mask


In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

text = "Héllo! How are you?"

# normalization فقط
normalized = tokenizer.backend_tokenizer.normalizer.normalize_str(text)
print(normalized)

# pipeline كامل
inputs = tokenizer(
    text,
    padding=True,
    return_tensors="pt"
)

print(inputs)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

hello! how are you?
{'input_ids': tensor([[ 101, 7592,  999, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


## 5️⃣ Load BLIP Model and Processor
هنحمل موديل image captioning مع الـ processor
## 6️⃣ Image Preprocessing
الـ processor بيعمل resize + normalization تلقائي
## 7️⃣ Generate Caption
نولد وصف للصورة باستخدام model.generate()

In [2]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)

processor = BlipProcessor.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)

from PIL import Image

image = Image.open("/kaggle/input/datasets/engmostafaali/photo0/Screenshot 2026-02-20 160652.png"
)

inputs = processor(image, return_tensors="pt")

output = model.generate(**inputs)

caption = processor.decode(output[0], skip_special_tokens=True)
print(caption)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/473 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie text_decoder.cls.predictions.bias to text_decoder.cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-base
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identic

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

a black dog running with a tennis ball


In [3]:
import os

for root, dirs, files in os.walk("/kaggle/input"):
    for file in files[:10]:
        print(os.path.join(root, file))

/kaggle/input/datasets/engmostafaali/photo0/Screenshot 2026-02-20 160652.png
/kaggle/input/datasets/engmostafaali/record/Vocaroo 21 Feb 2026 00_21_57 GMT0200 1cHW9qkT5oib.mp3


## 8️⃣ Load Audio Dataset
تحميل dataset وتجهيز الـ sampling rate
## 9️⃣ Audio Feature Extraction
تحويل waveform إلى input features
## 🔟 Prepare Audio for Model
تجهيز الصوت عشان يدخل موديل Whisper

In [4]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# حمّل الموديل
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# حمّل الصوت
audio_path = "/kaggle/input/datasets/engmostafaali/record/Vocaroo 21 Feb 2026 00_21_57 GMT0200 1cHW9qkT5oib.mp3"  # غير الاسم حسب ملفك
speech_array, sampling_rate = librosa.load(audio_path, sr=16000)

# حوله لمدخلات للموديل
inputs = processor(
    speech_array,
    sampling_rate=16000,
    return_tensors="pt"
)

# اعمل inference
with torch.no_grad():
    predicted_ids = model.generate(inputs["input_features"])

# فك التشفير
transcription = processor.batch_decode(
    predicted_ids,
    skip_special_tokens=True
)[0]

print("Transcription:", transcription)

preprocessor_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.log

Transcription:  السلام عليكم ورحمة الله وبركاته كان ياما كان


# ✅ Summary

- Text → Token IDs
- Image → Normalized Pixel Tensor
- Audio → Spectrogram Features

كلهم بيتحولوا لـ Tensors تدخل Transformer Model

In [5]:
from transformers import pipeline
pipe = pipeline(
    task="text-to-audio",
    model="facebook/musicgen-small",
)

result = pipe(
    "calm piano music",
    generate_kwargs={
        "temperature": 0.7,
        "max_new_tokens": 256
    }
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/611 [00:00<?, ?it/s]

MusicgenForConditionalGeneration LOAD REPORT from: facebook/musicgen-small
Key                                           | Status     |  | 
----------------------------------------------+------------+--+-
decoder.model.decoder.embed_positions.weights | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'return_dict_in_generate', 'temperature', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [7]:
from evaluate import combine

metrics = combine(["accuracy", "precision", "recall", "f1"])

results = metrics.compute(
    predictions=[0,1,1,0],
    references=[0,1,0,0]
)

print(results)

{'accuracy': 0.75, 'precision': 0.5, 'recall': 1.0, 'f1': 0.6666666666666666}
